In [45]:
# https://www.kaggle.com/aharless/xgboost-cv-lb-284?scriptVersionId=1683906

In [46]:
import sys
sys.path.insert(0, '/home/watts/Software/xgboost')

In [47]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import StratifiedKFold
from hccEncoding.EncoderForClassification import BayesEncoding,BayesEncodingKfold,LOOEncoding,LOOEncodingKfold

In [48]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [49]:
# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


In [50]:
def gini1(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

# def normalized_gini(solution, submission):
#     normalized_gini = gini1(solution, submission)/gini1(solution, solution)
#     return normalized_gini

In [51]:
def normalized_gini(solution, submission):
    normalized_gini = eval_gini(solution, submission)/eval_gini(solution, solution)
    return normalized_gini

In [52]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
     assert( len(actual) == len(pred) )
     all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
     all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
     totalLosses = all[:,0].sum()
     giniSum = all[:,0].cumsum().sum() / totalLosses
 
     giniSum -= (len(actual) + 1) / 2.
     return giniSum / len(actual)
 
def gini_normalized(a, p):
    return eval_gini(a, p) / eval_gini(a, a)

In [53]:
# Normalized Gini Scorer
# gini_scorer = make_scorer(normalized_gini, greater_is_better = True)

gini_scorer = make_scorer(gini_normalized, greater_is_better = True)

In [54]:
# from olivier
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
    "ps_reg_03",  #            : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
    "ps_ind_03",  #            : 1219.47 / shadow  230.55
    "ps_ind_15",  #            :  922.18 / shadow  242.00
    "ps_reg_02",  #            :  920.65 / shadow  267.50
    "ps_car_14",  #            :  798.48 / shadow  549.58
    "ps_car_12",  #            :  731.93 / shadow  293.62
    "ps_car_01_cat",  #        :  698.07 / shadow  178.72
    "ps_car_07_cat",  #        :  694.53 / shadow   36.35
    "ps_ind_17_bin",  #        :  620.77 / shadow   23.15
    "ps_car_03_cat",  #        :  611.73 / shadow   50.67
    "ps_reg_01",  #            :  598.60 / shadow  178.57
    "ps_car_15",  #            :  593.35 / shadow  226.43
    "ps_ind_01",  #            :  547.32 / shadow  154.58
    "ps_ind_16_bin",  #        :  475.37 / shadow   34.17
    "ps_ind_07_bin",  #        :  435.28 / shadow   28.92
    "ps_car_06_cat",  #        :  398.02 / shadow  212.43
    "ps_car_04_cat",  #        :  376.87 / shadow   76.98
    "ps_ind_06_bin",  #        :  370.97 / shadow   36.13
    "ps_car_09_cat",  #        :  214.12 / shadow   81.38
    "ps_car_02_cat",  #        :  203.03 / shadow   26.67
    "ps_ind_02_cat",  #        :  189.47 / shadow   65.68
    "ps_car_11",  #            :  173.28 / shadow   76.45
    "ps_car_05_cat",  #        :  172.75 / shadow   62.92
    "ps_calc_09",  #           :  169.13 / shadow  129.72
    "ps_calc_05",  #           :  148.83 / shadow  120.68
    "ps_ind_08_bin",  #        :  140.73 / shadow   27.63
    "ps_car_08_cat",  #        :  120.87 / shadow   28.82
    "ps_ind_09_bin",  #        :  113.92 / shadow   27.05
    "ps_ind_04_cat",  #        :  107.27 / shadow   37.43
    "ps_ind_18_bin",  #        :   77.42 / shadow   25.97
    "ps_ind_12_bin",  #        :   39.67 / shadow   15.52
    "ps_ind_14",  #            :   37.37 / shadow   16.65
]

In [55]:
# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [56]:
# Read data
df_train = pd.read_csv('../cache/train_X.csv') 
df_test = pd.read_csv('../cache/test_Y.csv')

In [57]:
df_train.isnull().sum()

ps_car_13                            0
ps_reg_03                       107772
ps_ind_05_cat                     5809
ps_ind_03                            0
ps_ind_15                            0
ps_reg_02                            0
ps_car_14                        42620
ps_car_12                            1
ps_car_01_cat                      107
ps_car_07_cat                    11489
ps_ind_17_bin                        0
ps_car_03_cat                   411231
ps_reg_01                            0
ps_car_15                            0
ps_ind_01                            0
ps_ind_16_bin                        0
ps_ind_07_bin                        0
ps_car_06_cat                        0
ps_car_04_cat                        0
ps_ind_06_bin                        0
ps_car_09_cat                      569
ps_car_02_cat                        5
ps_ind_02_cat                      216
ps_car_11                            5
ps_car_05_cat                   266551
ps_calc_09               

In [58]:
df_test.isnull().sum()

ps_car_13                            0
ps_reg_03                       161684
ps_ind_05_cat                     8710
ps_ind_03                            0
ps_ind_15                            0
ps_reg_02                            0
ps_car_14                        63805
ps_car_12                            0
ps_car_01_cat                      160
ps_car_07_cat                    17331
ps_ind_17_bin                        0
ps_car_03_cat                   616911
ps_reg_01                            0
ps_car_15                            0
ps_ind_01                            0
ps_ind_16_bin                        0
ps_ind_07_bin                        0
ps_car_06_cat                        0
ps_car_04_cat                        0
ps_ind_06_bin                        0
ps_car_09_cat                      877
ps_car_02_cat                        5
ps_ind_02_cat                      307
ps_car_11                            1
ps_car_05_cat                   400359
ps_calc_09               

In [59]:
df_train.fillna(-999, inplace=True)

In [60]:
df_test.fillna(-999, inplace=True)

In [61]:
df_train.head()

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,...,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat,id,target
0,0.883679,0.718070,0.0,5,11,0.2,0.370810,0.400000,10.0,1.0,...,0,0,1.0,0,0,0,19,70,7,0
1,0.618817,0.766078,0.0,7,3,0.4,0.388716,0.316228,11.0,1.0,...,1,0,0.0,1,0,0,21,80,9,0
2,0.641586,-999.000000,0.0,9,12,0.0,0.347275,0.316228,7.0,1.0,...,1,0,1.0,0,0,0,1,0,13,0
3,0.542949,0.580948,0.0,2,8,0.2,0.294958,0.374166,7.0,1.0,...,1,0,0.0,0,0,0,23,90,16,0
4,0.565832,0.840759,0.0,0,9,0.6,0.365103,0.316070,11.0,1.0,...,1,0,1.0,0,0,0,19,70,17,0


In [62]:
df_test.head()

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,...,ps_calc_05,ps_ind_08_bin,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat
0,0.669556,0.610328,0.0,8,12,0.3,0.352136,0.316228,7.0,1.0,...,1,0,1,0,1.0,0,0,0,14,50
1,0.606320,0.771362,0.0,5,5,0.5,0.358329,0.316228,4.0,1.0,...,3,0,1,1,1.0,0,0,0,23,90
2,0.896239,0.916174,0.0,3,10,0.0,0.398497,0.400000,11.0,1.0,...,3,0,1,1,0.0,0,0,0,11,40
3,0.652110,-999.000000,0.0,6,4,0.2,0.381445,0.374166,7.0,1.0,...,1,0,1,0,0.0,0,0,0,3,10
4,0.812914,0.817771,0.0,7,4,0.4,0.385097,0.374166,11.0,1.0,...,4,0,1,1,0.0,0,0,0,23,90


In [63]:
f_cats = [f for f in df_train.columns if "_cat" in f]

In [64]:
df = pd.read_csv('../cache/test_id.csv')

In [65]:
df.head()

,id
0,0
1,1
2,2
3,3
4,4


In [66]:
id_test = df['id'].values
id_train = df_train['id'].values
y = df_train['target'].values

In [67]:
sum(y == 0)

573518

In [68]:
sum(y==1)

21694

In [69]:
sp = sum(y==0)/sum(y==1)

In [70]:
sp

26.436710611228911

In [71]:
f_cats

['ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_07_cat',
 'ps_car_03_cat',
 'ps_car_06_cat',
 'ps_car_04_cat',
 'ps_car_09_cat',
 'ps_car_02_cat',
 'ps_ind_02_cat',
 'ps_car_05_cat',
 'ps_car_08_cat',
 'ps_ind_04_cat',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_reg_01_plus_ps_car_04_cat']

In [72]:
import category_encoders as ce

# encoder = ce.BackwardDifferenceEncoder(cols=f_cats)
# encoder = ce.BinaryEncoder(cols=f_cats)
# encoder = ce.HashingEncoder(cols=f_cats)
# encoder = ce.HelmertEncoder(cols=f_cats)
# encoder = ce.OneHotEncoder(cols=f_cats)
# encoder = ce.OrdinalEncoder(cols=f_cats)
encoder = ce.SumEncoder(cols=f_cats)
# encoder = ce.PolynomialEncoder(cols=f_cats)

encoder.fit(df_train, y)
df_train = encoder.transform(df_train)


In [73]:
# for f_cat in f_cats:    
#     df_train,df_test = BayesEncoding(df_train,df_test,'target',f_cat,f=1,noise=0.01)
#     df_train,df_test = LOOEncoding(df_train,df_test,'target',f_cat, noise=0.02)
#     df_train,df_test = LOOEncodingKfold(df_train,df_test,'target',f_cat, noise=0.01)

In [74]:
df_train.shape

(595212, 225)

In [75]:
df_test.shape

(892816, 36)

In [76]:
df_train.head()

,col_ps_ind_05_cat_0,col_ps_ind_05_cat_1,col_ps_ind_05_cat_2,col_ps_ind_05_cat_3,col_ps_ind_05_cat_4,col_ps_ind_05_cat_5,col_ps_ind_05_cat_6,col_ps_ind_05_cat_7,col_ps_car_01_cat_0,col_ps_car_01_cat_1,...,col_ps_car_11,col_ps_calc_09,col_ps_calc_05,col_ps_ind_08_bin,col_ps_ind_09_bin,col_ps_ind_18_bin,col_ps_ind_12_bin,col_ps_ind_14,col_id,col_target
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1,1,0,0,0,0,0,7,0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.0,1,1,1,0,1,0,0,9,0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,2,2,1,0,0,0,0,13,0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,4,4,0,0,0,0,0,16,0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.0,2,2,0,0,0,0,0,17,0


In [77]:
# for f in f_cats:
#     df_train = df_train.drop(f, axis=1)
#     df_test = df_test.drop(f, axis=1)

In [78]:
df_train.shape

(595212, 225)

In [79]:
df_test.shape

(892816, 36)

In [81]:
df_train = df_train.drop('col_id', axis=1)
df_train = df_train.drop('col_target', axis=1)

In [82]:
df_train.shape

(595212, 223)

In [83]:
X = df_train

In [84]:
X.isnull().sum()

col_ps_ind_05_cat_0                    0
col_ps_ind_05_cat_1                    0
col_ps_ind_05_cat_2                    0
col_ps_ind_05_cat_3                    0
col_ps_ind_05_cat_4                    0
col_ps_ind_05_cat_5                    0
col_ps_ind_05_cat_6                    0
col_ps_ind_05_cat_7                    0
col_ps_car_01_cat_0                    0
col_ps_car_01_cat_1                    0
col_ps_car_01_cat_2                    0
col_ps_car_01_cat_3                    0
col_ps_car_01_cat_4                    0
col_ps_car_01_cat_5                    0
col_ps_car_01_cat_6                    0
col_ps_car_01_cat_7                    0
col_ps_car_01_cat_8                    0
col_ps_car_01_cat_9                    0
col_ps_car_01_cat_10                   0
col_ps_car_01_cat_11                   0
col_ps_car_01_cat_12                   0
col_ps_car_07_cat_0                    0
col_ps_car_07_cat_1                    0
col_ps_car_07_cat_2                    0
col_ps_car_03_ca

In [85]:
df_test.isnull().sum()

ps_car_13                       0
ps_reg_03                       0
ps_ind_05_cat                   0
ps_ind_03                       0
ps_ind_15                       0
ps_reg_02                       0
ps_car_14                       0
ps_car_12                       0
ps_car_01_cat                   0
ps_car_07_cat                   0
ps_ind_17_bin                   0
ps_car_03_cat                   0
ps_reg_01                       0
ps_car_15                       0
ps_ind_01                       0
ps_ind_16_bin                   0
ps_ind_07_bin                   0
ps_car_06_cat                   0
ps_car_04_cat                   0
ps_ind_06_bin                   0
ps_car_09_cat                   0
ps_car_02_cat                   0
ps_ind_02_cat                   0
ps_car_11                       0
ps_car_05_cat                   0
ps_calc_09                      0
ps_calc_05                      0
ps_ind_08_bin                   0
ps_car_08_cat                   0
ps_ind_09_bin 

In [86]:
sp

26.436710611228911

In [87]:
xgb_model = XGBClassifier(objective='binary:logistic', missing=-999)
parameters = {
    'learning_rate': [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1], #so called `eta` value
    'max_depth': sp_randint(1,11),
    'min_child_weight': sp_randint(1,11),
    'silent': [1],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.8, 0.7],
    'scale_pos_weight': [26.4367],
    'gamma': sp_randint(0,15),
    'reg_alpha': sp_randint(0,10),
    'reg_lambda': [0, 1, 1.3,1.5, 2.0, 3.0],
    'missing':[-999],
    'seed': [1337]
}


clf =  RandomizedSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, 5),
                   scoring = gini_scorer,
                   verbose=3, refit=True)


In [88]:
clf.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] seed=1337, gamma=3, silent=1, subsample=0.8, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=7, max_depth=5, colsample_bytree=0.7, reg_lambda=1 
[CV] seed=1337, gamma=3, silent=1, subsample=0.8, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=7, max_depth=5, colsample_bytree=0.7, reg_lambda=1 
[CV] seed=1337, gamma=3, silent=1, subsample=0.8, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=7, max_depth=5, colsample_bytree=0.7, reg_lambda=1 
[CV] seed=1337, gamma=3, silent=1, subsample=0.8, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=7, max_depth=5, colsample_bytree=0.7, reg_lambda=1 
[CV] seed=1337, gamma=3, silent=1, subsample=0.8, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=7, max_depth=5, colsample_bytree=0.7

[CV] seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8, reg_lambda=1.5 
[CV]  seed=1337, gamma=5, silent=1, subsample=0.7, min_child_weight=1, learning_rate=0.05, scale_pos_weight=26.4367, missing=-999, reg_alpha=0, max_depth=9, colsample_bytree=0.8, reg_lambda=1.3, score=0.154041 -16.5min
[CV] seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8, reg_lambda=1.5 
[CV]  seed=1337, gamma=5, silent=1, subsample=0.7, min_child_weight=1, learning_rate=0.05, scale_pos_weight=26.4367, missing=-999, reg_alpha=0, max_depth=9, colsample_bytree=0.8, reg_lambda=1.3, score=0.166799 -16.7min
[CV] seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8,

[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed: 52.5min


[CV]  seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8, reg_lambda=1.5, score=0.181512 -14.6min
[CV] seed=1337, gamma=10, silent=1, subsample=0.7, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=1, max_depth=7, colsample_bytree=0.8, reg_lambda=1 
[CV]  seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8, reg_lambda=1.5, score=0.161557 -14.6min
[CV] seed=1337, gamma=10, silent=1, subsample=0.7, min_child_weight=1, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=1, max_depth=7, colsample_bytree=0.8, reg_lambda=1 
[CV]  seed=1337, gamma=9, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=9, max_depth=8, colsample_bytree=0.8, re

[CV]  seed=1337, gamma=9, silent=1, subsample=0.8, min_child_weight=10, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=6, max_depth=5, colsample_bytree=0.8, reg_lambda=2.0, score=0.199948 - 8.5min
[CV] seed=1337, gamma=14, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.08, scale_pos_weight=26.4367, missing=-999, reg_alpha=8, max_depth=9, colsample_bytree=0.8, reg_lambda=3.0 
[CV]  seed=1337, gamma=9, silent=1, subsample=0.8, min_child_weight=10, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=6, max_depth=5, colsample_bytree=0.8, reg_lambda=2.0, score=0.200839 - 8.8min
[CV] seed=1337, gamma=14, silent=1, subsample=0.7, min_child_weight=4, learning_rate=0.08, scale_pos_weight=26.4367, missing=-999, reg_alpha=8, max_depth=9, colsample_bytree=0.8, reg_lambda=3.0 
[CV]  seed=1337, gamma=9, silent=1, subsample=0.8, min_child_weight=10, learning_rate=0.1, scale_pos_weight=26.4367, missing=-999, reg_alpha=6, max_depth=5, colsample_bytre

[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed: 109.0min finished


RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=5, shuffle=False, random_state=None),
          error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=-999, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=10, n_jobs=5,
          param_distributions={'seed': [1337], 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7395b88160>, 'silent': [1], 'subsample': [0.8, 0.7], 'min_child_weight': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7395b88128>, 'learning_rate': [0.03, 0.04, 0.05, 0.06, 0.07, 0....object at 0x7f73dc9ed390>, 'colsample_bytree': [0.8, 0.7], 

In [89]:
print(clf.best_score_)
print(clf.best_params_)

# 0.24184412440294284
# {'reg_lambda': 1, 'reg_alpha': 9, 'seed': 1337, 'gamma': 13, 'max_depth': 8, 'silent': 1, 
#  'subsample': 0.7, 'min_child_weight': 4, 'learning_rate': 0.08, 'colsample_bytree': 0.8, 'missing': -999}
# 0.254 on public LB,

# 0.2382713543302345
# {'seed': 1337, 'max_depth': 5, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 5, 
#  'learning_rate': 0.08, 'colsample_bytree': 0.7, 'missing': -999}
# when gamma, reg_alpha, reg_lamda is filetered out, using new gini scorer

# 0.23755666055779237
# {'seed': 1337, 'gamma': 6, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 6, 
#  'learning_rate': 0.1, 'colsample_bytree': 0.7, 'missing': -999, 'reg_alpha': 8, 'max_depth': 9, 
#  'scale_pos_weight': 26.4367, 'reg_lambda': 3.0}
# putting gamma back, shifting scale_pos_weightin parameters, using new gini scorer

# using f =10 and noise=0 in the encoding functions
# 1.0
# {'seed': 1337, 'gamma': 2, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 5, 
#  'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}

# using f =10 and noise=0 in the encoding functions, dropped LOO
# 0.203635367561805
# {'seed': 1337, 'gamma': 5, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'missing': -999, 
#  'reg_alpha': 7, 'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 2.0}

# using f = 20, noise=0.0, no LOO
# 0.1982897319930031
# {'seed': 1337, 'gamma': 1, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 1, 'learning_rate': 0.06, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 6, 
#  'max_depth': 6, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.3}

# f = 10, noise=0.01 no LOO
# 0.1921171790111911
# {'seed': 1337, 'gamma': 2, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.7, 'missing': -999, 
#  'reg_alpha': 9, 'max_depth': 6, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}

# f = 1, noise=0.01, no LOO
#0.19488331448594218
# {'seed': 1337, 'gamma': 5, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 3, 'learning_rate': 0.06, 'colsample_bytree': 0.7, 'missing': -999, 'reg_alpha': 4, 'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}
# In [45]:

# LOO only, noise=0
# 1.0
# {'seed': 1337, 'gamma': 0, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 8, 'learning_rate': 0.08, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 7, 
#  'max_depth': 8, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}

# LOO only , noise=0.01, last gini
# 0.2572973729916049
# {'seed': 1337, 'gamma': 8, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'missing': -999, 'reg_alpha': 0, 
#  'max_depth': 6, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.5}
# scored 0.264 on pubic LB

# LOO only, noise=0.01, with eval_gini
# 0.2572973729916049
# {'seed': 1337, 'gamma': 8, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'missing': -999, 
#  'reg_alpha': 0, 'max_depth': 6, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.5}

# LOO only, noise=0.02, with eval_gini
# 0.212160199299836
# {'seed': 1337, 'gamma': 11, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 5, 'learning_rate': 0.07, 'colsample_bytree': 0.7, 'missing': -999, 'reg_alpha': 2, 
#  'max_depth': 5, 'scale_pos_weight': 26.4367, 'reg_lambda': 3.0}

# LOOKFold, noise=0.01, eval gini
# 0.15067528137467348
# {'seed': 1337, 'gamma': 14, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 8, 'learning_rate': 0.04, 'colsample_bytree': 0.8, 'missing': -999, 
#  'reg_alpha': 2, 'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 1}

# Binary Encoder
# 0.196550404720537
# {'seed': 1337, 'gamma': 8, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 6, 'learning_rate': 0.06, 'colsample_bytree': 0.7, 'missing': -999, 
#  'reg_alpha': 4, 'max_depth': 5, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}

# Hashing Encoder
# 0.18163349897449646
# {'seed': 1337, 'gamma': 1, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.7, 'missing': -999, 'reg_alpha': 7, 
#  'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.5}

# helmert
# 0.1998236311643095
# {'seed': 1337, 'gamma': 4, 'silent': 1, 'subsample': 0.8, 'min_child_weight': 8, 'learning_rate': 0.07, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 6, 
#  'max_depth': 5, 'scale_pos_weight': 26.4367, 'reg_lambda': 0}

# OneHot
# 0.1974928419377269
# {'seed': 1337, 'gamma': 10, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 1, 'learning_rate': 0.06, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 7, 
#  'max_depth': 6, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.5}

#ordinal
# 1.0
#stopped

# Polynomial encoder
# 0.19832716986326504
# {'seed': 1337, 'gamma': 7, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 5, 'learning_rate': 0.06, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 1,
#  'max_depth': 4, 'scale_pos_weight': 26.4367, 'reg_lambda': 1}

# Sum encoder
# 0.1981555069566366
# {'seed': 1337, 'gamma': 7, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 1, 'learning_rate': 0.05, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 9, 
#  'max_depth': 7, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.3}

0.1981555069566366
{'seed': 1337, 'gamma': 7, 'silent': 1, 'subsample': 0.7, 'min_child_weight': 1, 'learning_rate': 0.05, 'colsample_bytree': 0.8, 'missing': -999, 'reg_alpha': 9, 'max_depth': 7, 'scale_pos_weight': 26.4367, 'reg_lambda': 1.3}


In [42]:
df_test.fillna(-999, inplace=True)

In [43]:
test_probs = clf.predict_proba(df_test)[:,1]

In [44]:
test_probs

array([ 0.37710699,  0.42687458,  0.43030238, ...,  0.45527622,
        0.42130691,  0.452133  ], dtype=float32)

In [45]:
test_probs.shape

(892816,)

In [46]:
np.min(test_probs)

0.054419678

In [47]:
np.max(test_probs)

0.98862666

In [48]:
test_probs = np.clip(test_probs, a_min=0.05, a_max=0.95)

In [49]:
submission = pd.DataFrame()
submission['id'] = id_test
submission['target'] = test_probs

submission.to_csv('../submissions/sub_xgb_2_0.csv', index=False)
